In [1]:
import sys
import csv
import pandas as pd
from collections import defaultdict

if len(sys.argv)<5:
    print('result and gold not specified. Default is ./opencc/simp2multitrad_test_opencc_s2, gold is ./test_cases/simp2multitrad_test.csv')
    result='./opencc/simp2multitrad_test_opencc_s2tw'
    gold='./test_cases/simp2multitrad_test_gold.csv'
else:
    result=sys.argv[1]
    gold=sys.argv[2]
    print (result,gold)



result and gold not specified. Default is ./opencc/simp2multitrad_test_opencc_s2, gold is ./test_cases/simp2multitrad_test.csv


In [8]:
gold_csv = pd.read_csv(gold)
gold_dict=gold_csv.to_dict('records')
gold_dict

[{'char_index': 25,
  'gold': '商務部統計顯示,4月份零售是繼3月份下跌1.3%之後的連續第二個月下跌。',
  'gold_char': '後',
  'orig': '商务部统计显示,4月份零售是继3月份下跌1.3%之后的连续第二个月下跌。',
  'orig_char': '后',
  'orig_line_num': 6400437},
 {'char_index': 1,
  'gold': '就後者而言,佘契爾夫人認為梅傑在一九九零年十月任財政大臣時,將英鎊帶入歐洲匯率機制,是因為受到當時「時髦的共識」的影響。',
  'gold_char': '後',
  'orig': '就后者而言,佘契尔夫人认为梅杰在一九九零年十月任财政大臣时,将英镑带入欧洲汇率机制,是因为受到当时「时髦的共识」的影响。',
  'orig_char': '后',
  'orig_line_num': 19086588},
 {'char_index': 25,
  'gold': '享年七十九歲的密特朗是在去年任滿離開總統職位八個月後,於今天上午因前列腺癌不治,病逝巴黎寓所。',
  'gold_char': '後',
  'orig': '享年七十九岁的密特朗是在去年任满离开总统职位八个月后,于今天上午因前列腺癌不治,病逝巴黎寓所。',
  'orig_char': '后',
  'orig_line_num': 12812637},
 {'char_index': 5,
  'gold': '與賈梅會談後,黃志芳中午宴請旅甘僑胞、台商及技術團團員,對他們在國外辛苦打拼表達謝意,並代表陳總統致贈慰問金。',
  'gold_char': '後',
  'orig': '与贾梅会谈后,黄志芳中午宴请旅甘侨胞、台商及技术团团员,对他们在国外辛苦打拼表达谢意,并代表陈总统致赠慰问金。',
  'orig_char': '后',
  'orig_line_num': 19933963},
 {'char_index': 8,
  'gold': '來自大陸漁船處理後的烏魚子鮮度差,低於台灣貨五、六百元。',
  'gold_char': '後',
  'orig': '来自大陆渔船处理后的乌鱼子鲜度差,低于台湾货五、六百元。',
  'orig_char

In [18]:


line_num=0
num_incor=0
num_corr=0
accu_per_char=defaultdict(float)
num_per_char=defaultdict(float)

print ('printing incorrect predictions')
print ('char_gold',":",'error_rate')
with open(result+'_error', 'w') as error_f:
    with open(result, 'r') as res_f:
        fieldnames = ['char_res','orig_char', 'gold_char','char_index','res','orig','gold','orig_line_num']
        writer = csv.DictWriter(error_f, fieldnames=fieldnames)

        writer.writeheader()
        for line in res_f:
            char_i=gold_csv['char_index'][line_num]
            char_gold=gold_csv['gold_char'][line_num]
            char_res=line[char_i]
            num_per_char[char_gold]+=1


            if char_gold!=char_res:
                gold_dict[line_num]['char_res']=char_res
                gold_dict[line_num]['res']=line
                writer.writerow(gold_dict[line_num])
                #print (char_gold,char_res,char_i,line_num,gold_csv['gold'][line_num])
                num_incor+=1
                accu_per_char[char_gold]+=1
            else:
                accu_per_char[char_gold]=accu_per_char[char_gold]
                num_corr+=1
            line_num+=1   
for key in accu_per_char:
    print (key, ': ', float(accu_per_char[key])/100)
print (num_incor, 'incorrect cases')
print(len(gold_csv))
print (float(num_incor/len(gold_csv)), 'incorrect. total 133000',)

printing incorrect predictions
char_gold : error_rate
後 :  0.0
發 :  0.0
台 :  1.0
家 :  0.0
出 :  0.0
表 :  0.0
了 :  0.04
當 :  0.0
劃 :  0.0
克 :  0.0
雲 :  0.0
鬆 :  0.01
鐘 :  0.17
復 :  0.06
郁 :  0.57
獲 :  0.0
徵 :  0.01
向 :  0.01
制 :  0.0
里 :  0.15
衝 :  0.0
須 :  0.0
面 :  0.0
致 :  0.01
儘 :  0.05
藉 :  0.08
繫 :  0.07
系 :  0.01
匯 :  0.03
準 :  0.0
複 :  0.15
係 :  0.11
姜 :  0.01
隻 :  0.04
歷 :  0.02
松 :  0.21
臺 :  0.0
谷 :  0.01
瞭 :  0.19
只 :  0.06
豐 :  0.0
回 :  0.11
幹 :  0.01
蒙 :  0.02
曆 :  0.03
簽 :  0.07
惡 :  0.0
製 :  0.12
板 :  0.02
據 :  0.0
盡 :  0.25
乾 :  0.15
餘 :  0.0
範 :  0.0
沈 :  0.03
余 :  0.57
范 :  0.62
麵 :  0.19
臟 :  0.0
胡 :  0.09
借 :  0.07
颱 :  0.12
檯 :  0.22
曲 :  0.0
築 :  0.0
干 :  0.06
折 :  0.02
准 :  0.23
彌 :  0.01
癥 :  0.23
闆 :  0.0
誇 :  0.0
迴 :  0.07
征 :  0.28
卷 :  0.0
症 :  0.01
舍 :  0.0
捨 :  0.11
鬥 :  0.0
傢 :  0.01
鍾 :  0.18
后 :  0.19
摺 :  0.16
穀 :  0.19
髮 :  0.04
醜 :  0.01
沖 :  0.21
禦 :  0.0
斗 :  0.05
裏 :  1.0
夥 :  0.23
据 :  0.39
伙 :  0.6
丰 :  0.35
髒 :  0.01
樸 :  0.05
朴 :  0.38
錶 :  0.2
